In [1]:
# modulos
import sympy as sp
import numpy as np

# simbolos generales
s,R1,R2,R3,R4,R5,R6,C1,C2,C3,C4,Vi,Vo,Vx,V,k,Vy=sp.symbols("s,R1,R2,R3,R4,R5,R6,C1,C2,C3,C4,Vi,Vo,Vx,V,k,Vy")

g1=1/R1
g2=1/R2
g3=1/R3
g4=1/R4
X2=1/(s*C2)

# print rapido de ecuaciones
def pp(eqs):
    print("\n")
    [sp.pprint(eq,wrap_line=False) for eq in eqs]
    print("\n")

# print rapido de separadores
def pr(txt=None):
    # si no hay texto, imprimir linea de fin
    if txt==None:
        print(f"{'*'*100}\n")
    
    # sino, imprimir un cartel de inicio
    else:
        tittle=f"|{txt}|".center(2*len(txt))
        
        print("-"*len(tittle))
        print(tittle)
        print("-"*len(tittle))


# Sallen-Key
!["sallenKey"](./img/sallenKeyBP.jpg)

In [2]:
# ecuacion de circuito

# print("-"*32)
# print("|\tEcuaciones de nodos.\t|")
# print("-"*32)
pr("Ecuaciones de nodos.")

nodo0=sp.Eq(Vx*(g1+g2+s*C1+s*C2)-Vo*g2-(Vo/k)*s*C2-g1*Vi,0)
nodo1=sp.Eq((Vo/k)*(s*C2+g3)-s*C2*Vx,0)
pp([nodo0,nodo1])

# sp.print_latex(nodo0)
# sp.print_latex(nodo1)

pr()

# -----------------------------------------------------------------------------------
# buscar funcion de transferencia
pr("Funcion de transferencia.")

# resolver para vo y vi, y dividir
sol0=sp.solve((nodo0,nodo1),(Vo,Vi))

T=sp.collect(sp.simplify(sol0[Vo]/sol0[Vi]),s)
pp([sp.Eq(Vo/Vi,T)])

pr()

# -----------------------------------------------------------------------------------
# normalizar 
pr("Normalizacion.")

# recuperar num den
n,d=sp.fraction(T)

# normalizar coeficiente principal del denominador
n=sp.simplify(n/d.coeff(s,2))
d=sp.simplify(d/d.coeff(s,2))

T=sp.collect(n/d,s)
pp([sp.Eq(sp.symbols("T"),T)])

# sp.print_latex(T)

pr()

----------------------------------------
         |Ecuaciones de nodos.|         
----------------------------------------


  C₂⋅Vo⋅s      ⎛              1    1 ⎞   Vo   Vi    
- ─────── + Vx⋅⎜C₁⋅s + C₂⋅s + ── + ──⎟ - ── - ── = 0
     k         ⎝              R₂   R₁⎠   R₂   R₁    
              ⎛       1 ⎞    
           Vo⋅⎜C₂⋅s + ──⎟    
              ⎝       R₃⎠    
-C₂⋅Vx⋅s + ────────────── = 0
                 k           


****************************************************************************************************

--------------------------------------------------
           |Funcion de transferencia.|            
--------------------------------------------------


Vo                                         C₂⋅R₂⋅R₃⋅k⋅s                                      
── = ────────────────────────────────────────────────────────────────────────────────────────
Vi                   2                                                                       
     C₁⋅C₂⋅R₁⋅R₂⋅R₃⋅s  + 

## Diseño

In [3]:
# requisotos
k1=5252.26 
a1=3184.42 
b1=66320656

# funciones de transferencia
pr("Transferencias")

T1=T

pp([sp.Eq(T1,k1*s/(s**2+a1*s+b1))])
# sp.print_latex(sp.Eq(T1,-k1*s/(s**2+a1*s+b1)))
pr()

# igualacion de coef
pr("Igualaciones")

# extraer y normalizar num y den
n1,d1=sp.fraction(T1)
cp=d1.expand().coeff(s,2)

ig1=sp.Eq((d1/cp).coeff(s,1),a1)
ig2=sp.Eq((d1/cp).coeff(s,0),b1)
pp([ig1,ig2])
pr()

# valuando componentes
pr("Valuando componentes")

c1=1
c2=1
R=sp.symbols("R")

ig1=ig1.subs({C1:c1,C2:c2,R1:R,R2:R,R3:R})
ig2=ig2.subs({C1:c1,C2:c2,R1:R,R2:R,R3:R})
pp([ig1,ig2])
# sp.print_latex(ig1)
# sp.print_latex(ig2)
pr()

# calculo de componentes
pr("Valores")

# ignorar parte negativa de las raices
(r1,k1)=sp.solve((ig1,ig2),(R,k))[1]

# componentes
escala1=10E6
escala2=10E6

pp([sp.Eq(C1,c1/escala1),sp.Eq(C2,c2/escala1),sp.Eq(R,r1*escala1),sp.Eq(k,k1)])

# sp.print_latex(sp.Eq(C1,c1/escala1))
# sp.print_latex(sp.Eq(C2,c2/escala1))
# sp.print_latex(sp.Eq(R,r1*escala1))
# sp.print_latex(sp.Eq(k,k1))
pr()

pr("valores de implementacion")
c1=100E-9
c2=100E-9
r1=1738
k1=3.44

----------------------------
      |Transferencias|      
----------------------------


                                       k⋅s                                                   5252.26⋅s        
────────────────────────────────────────────────────────────────────────────────── = ─────────────────────────
      ⎛ 2     ⎛  1       1       k       1       1  ⎞        1             1     ⎞    2                       
C₁⋅R₁⋅⎜s  + s⋅⎜───── + ───── - ───── + ───── + ─────⎟ + ─────────── + ───────────⎟   s  + 3184.42⋅s + 66320656
      ⎝       ⎝C₂⋅R₃   C₁⋅R₃   C₁⋅R₂   C₁⋅R₂   C₁⋅R₁⎠   C₁⋅C₂⋅R₂⋅R₃   C₁⋅C₂⋅R₁⋅R₃⎠                            


****************************************************************************************************

------------------------
     |Igualaciones|     
------------------------


  1       1       k       1       1            
───── + ───── - ───── + ───── + ───── = 3184.42
C₂⋅R₃   C₁⋅R₃   C₁⋅R₂   C₁⋅R₂   C₁⋅R₁          
     1             1           

## Sensitividades

In [4]:
pr("Simplificaciones")

# simbolos auxiliares
w1,w12,w2,w21,w31,w32=sp.symbols("w1,w12,w2,w21,w31,w32")

# simplificar ft
T0=T.subs({1/(R1*C1):w1,1/(R2*C2):w2}).subs({1/(R1*C2):w12,
                                             1/(R2*C1):w21,
                                             1/(R3*C1):w31,
                                             1/(R3*C2):w32})
pp([sp.Eq(T,T0)])

pr()

# --------------------------------------------------------------------------------------
pr("Parametros de analisis")
n0,d0=sp.fraction(T0)

wp2=sp.factor(d0.coeff(s,0))
bwp=sp.factor(d0.coeff(s,1))

pp([sp.Eq(sp.sympify("wp**2"),wp2),sp.Eq(sp.sympify("bwp"),bwp)])

pr()

--------------------------------
       |Simplificaciones|       
--------------------------------


                                       k⋅s                                                                    k⋅s⋅w₁                         
────────────────────────────────────────────────────────────────────────────────── = ────────────────────────────────────────────────────────
      ⎛ 2     ⎛  1       1       k       1       1  ⎞        1             1     ⎞    2                                                      
C₁⋅R₁⋅⎜s  + s⋅⎜───── + ───── - ───── + ───── + ─────⎟ + ─────────── + ───────────⎟   s  + s⋅(-k⋅w₂₁ + w₁ + w₂₁ + w₃₁ + w₃₂) + w₁⋅w₃₂ + w₂⋅w₃₁
      ⎝       ⎝C₂⋅R₃   C₁⋅R₃   C₁⋅R₂   C₁⋅R₂   C₁⋅R₁⎠   C₁⋅C₂⋅R₂⋅R₃   C₁⋅C₂⋅R₁⋅R₃⎠                                                           


****************************************************************************************************

--------------------------------------------
          |Parametros de analisis|      

### Sensitividades Wp:

In [21]:
# -----------------------------------------------------------------------------------------
pr("Sensitividad para wp.")
pp([sp.Eq(sp.sympify("wp**2"),wp2)])
sp.print_latex(sp.Eq(sp.sympify("wp**2"),wp2))

""" wp2 == w₁⋅w₃₂ + w₂⋅w₃₁ """

""" entender la operatoria para un parametro, por ej R1 """
# w1 == 1/(R1*C1) == (R1*C1)**-1
# w1ssR1=-1
# w32ssR1=0
# w2ssR1=0
# w31ssR1=0

# propiedad: (p1*p2)Sx == p1Sx + p2Sx
# w1w32ssR1=w1ssR1+w32ssR1
# w2w31ssR1=w2ssR1+w31ssR1

# propiedad: (p1+p2)Sx == (p1*p1Sx + p2*p2Sx) / (p1+p2)
# wp2ssR1=(w1*w32*w1w32ssR1 + w2*w31*w2w31ssR1)/(w1*w32 + w2*w31)

# propiedad: (p**n)Sx == n*(pSx)
# wpssR1=wp2ssR1/2

""" y generalizar la operatoria para cualquier parametro"""
# formato: pssx == [pssR1, pssR2,... ,pssRn, pssC1, pssC2,... , pssCn]
w1ssx=np.array([-1,0,0,-1,0])
w32ssx=np.array([0,0,-1,0,-1])
w2ssx=np.array([0,-1,0,0,-1])
w31ssx=np.array([0,0,-1,-1,0])

# propiedad: (p1*p2)Sx == p1Sx + p2Sx
w1w32ssx=w1ssx+w32ssx
w2w31ssx=w2ssx+w31ssx

# propiedad: (p1+p2)Sx == (p1*p1Sx + p2*p2Sx) / (p1+p2)
wp2ssx=(w1*w32*w1w32ssx + w2*w31*w2w31ssx)/(w1*w32 + w2*w31)

# propiedad: (p**n)Sx == n*(pSx)
wpssx=wp2ssx/2

# generar un diccionario
keys=[R1,R2,R3,C1,C2]

# aux={}
# for kk in range(len(keys)):
#     aux.update({keys[kk]:wpssx[kk]})
wpStv={k:sp.simplify(v.subs({w1:1/(R1*C1),
                             w2:1/(R2*C2),
                             w12:1/(R1*C2),
                             w21:1/(R2*C1),
                             w31:1/(R3*C1),
                             w32:1/(R3*C2)})) for k,v in zip(keys,wpssx)}
pp([wpStv])
sp.print_latex(wpStv)

# comprobar los calculos con identidad para wp
# identidad: sum(wpssC) == sum(wpssR) == -1
wpSumR=sp.simplify(wpssx[0]+wpssx[1]+wpssx[2])
wpSumC=sp.simplify(wpssx[3]+wpssx[4])
pp([sp.Eq(sp.Sum(sp.symbols("wpSsR_i"), (sp.symbols("i"), 1, 3)),wpSumR)])
pp([sp.Eq(sp.Sum(sp.symbols("wpSsC_i"), (sp.symbols("i"), 1, 2)),wpSumC)])
sp.print_latex(sp.Eq(sp.Sum(sp.symbols("wpSsR_i"), (sp.symbols("i"), 1, 3)),wpSumR))
sp.print_latex(sp.Eq(sp.Sum(sp.symbols("wpSsC_i"), (sp.symbols("i"), 1, 2)),wpSumC))
pr()

pr("Sensibilidades numericas")

# obtener valores numericos
NUMwpStv={k:wpStv[k].subs({R1:r1,R2:r1,R3:r1,C1:c1,C2:c2}) for k in keys}

pp([NUMwpStv])

------------------------------------------
         |Sensitividad para wp.|          
------------------------------------------


  2                  
wp  = w₁⋅w₃₂ + w₂⋅w₃₁


\wp^{2} = w_{1} w_{32} + w_{2} w_{31}


⎧                            -R₂              -R₁              ⎫
⎨C₁: -1/2, C₂: -1/2, R₁: ───────────, R₂: ───────────, R₃: -1/2⎬
⎩                        2⋅R₁ + 2⋅R₂      2⋅R₁ + 2⋅R₂          ⎭


\left\{ C_{1} : - \frac{1}{2}, \  C_{2} : - \frac{1}{2}, \  R_{1} : - \frac{R_{2}}{2 R_{1} + 2 R_{2}}, \  R_{2} : - \frac{R_{1}}{2 R_{1} + 2 R_{2}}, \  R_{3} : - \frac{1}{2}\right\}


  3              
 ___             
 ╲               
  ╲              
  ╱   wpSsRᵢ = -1
 ╱               
 ‾‾‾             
i = 1            




  2              
 ___             
 ╲               
  ╲              
  ╱   wpSsCᵢ = -1
 ╱               
 ‾‾‾             
i = 1            


\sum_{i=1}^{3} wpSsR_{i} = -1
\sum_{i=1}^{2} wpSsC_{i} = -1
************************************************

### Sensitividades bwp:

In [26]:
# -----------------------------------------------------------------------------------------
pr("Sensitividad para bwp.")
pp([sp.Eq(sp.sympify("bwp"),bwp)])
sp.print_latex(sp.Eq(sp.sympify("bwp"),bwp))

""" bwp == -k⋅w₂₁ + w₁ + w₂₁ + w₃₁ + w₃₂ == w₂₁⋅(1-k) + w₁ + w₃₁ + w₃₂ """ 
# sensitividades basicas
w1ssx=w1ssx
w21ssx=np.array([0,-1,0,-1,0])
w31ssx=w31ssx
w32ssx=w32ssx

# propiedad: (cte*p)Sx == pSx
cte_w21ssx=w21ssx

# propiedad: (p1+p2+ ...+pn)Sx == (p1*p1Sx + p2*p2Sx + ...+ pn*pnSx) / (p1+p2+ ...+pn)
bwpssx=((1-k)*w21*cte_w21ssx + w1*w1ssx + w31*w31ssx + w32*w32ssx)/(bwp)

# generar un diccionario
keys=[R1,R2,R3,C1,C2]

bwpStv={k:sp.simplify(v.subs({w1:1/(R1*C1),
                              w2:1/(R2*C2),
                              w12:1/(R1*C2),
                              w21:1/(R2*C1),
                              w31:1/(R3*C1),
                              w32:1/(R3*C2),
                              bwp:sp.symbols("bwp")})) for k,v in zip(keys,bwpssx)}

pp([bwpStv])
sp.print_latex(bwpStv)

# comprobar los calculos con identidad para Qp
# identidad: sum(QpssC) == sum(QpssR) == 0

""" Aca se hace de forma indirecta porque tenemos las bwpSx pero no las QpSx. Pero aplicando propiedades se hace sencillo: """
# bwp == wp/Qp
# propiedad: (p1/p2)Sx == p1Sx - p2Sx
# bwpSx == (wp/Qp)Sx == wpSx - QpSx ==> QpSx == wpSx - bwpSx (calculadas)

QpSumR=sp.simplify((wpssx[0]-bwpssx[0])+(wpssx[1]-bwpssx[1])+(wpssx[2]-bwpssx[2]))
QpSumC=sp.simplify((wpssx[3]-bwpssx[3])+(wpssx[4]-bwpssx[4]))
pp([sp.Eq(sp.Sum(sp.symbols("QpSsR_i"), (sp.symbols("i"), 1, 3)),QpSumR)])
pp([sp.Eq(sp.Sum(sp.symbols("QpSsC_i"), (sp.symbols("i"), 1, 2)),QpSumC)])
sp.print_latex(sp.Eq(sp.Sum(sp.symbols("QpSsR_i"), (sp.symbols("i"), 1, 3)),QpSumR))
sp.print_latex(sp.Eq(sp.Sum(sp.symbols("QpSsC_i"), (sp.symbols("i"), 1, 2)),QpSumC))
pr()

pr("Sensibilidades numericas")

# obtener valores numericos
NUMbwpStv={kk:bwpStv[kk].subs({R1:r1,R2:r1,R3:r1,C1:c1,C2:c2,k:k1,sp.symbols('bwp'):a1}) for kk in keys}

pp([NUMbwpStv])    

--------------------------------------------
          |Sensitividad para bwp.|          
--------------------------------------------


bwp = -k⋅w₂₁ + w₁ + w₂₁ + w₃₁ + w₃₂


bwp = - k w_{21} + w_{1} + w_{21} + w_{31} + w_{32}


⎧    -R₁⋅R₂ - R₁⋅R₃⋅(1 - k) - R₂⋅R₃         -1             -1            k - 1          -C₁ - C₂  ⎫
⎨C₁: ──────────────────────────────, C₂: ─────────, R₁: ─────────, R₂: ─────────, R₃: ────────────⎬
⎩           C₁⋅R₁⋅R₂⋅R₃⋅bwp              C₂⋅R₃⋅bwp      C₁⋅R₁⋅bwp      C₁⋅R₂⋅bwp      C₁⋅C₂⋅R₃⋅bwp⎭


\left\{ C_{1} : \frac{- R_{1} R_{2} - R_{1} R_{3} \cdot \left(1 - k\right) - R_{2} R_{3}}{C_{1} R_{1} R_{2} R_{3} bwp}, \  C_{2} : - \frac{1}{C_{2} R_{3} bwp}, \  R_{1} : - \frac{1}{C_{1} R_{1} bwp}, \  R_{2} : \frac{k - 1}{C_{1} R_{2} bwp}, \  R_{3} : \frac{- C_{1} - C_{2}}{C_{1} C_{2} R_{3} bwp}\right\}


  3             
 ___            
 ╲              
  ╲             
  ╱   QpSsRᵢ = 0
 ╱              
 ‾‾‾            
i = 1           




  2             
 __

# Desviaciones

In [32]:
""" Dp/p == pSsR1*DR1/R1 + pSsR2*DR2/R2 + pSsR3*DR3/R3 + pSsC1*DC1/C1 + pSsC2*DC2/C2 == (VvR)*SUM[pSsRi] + (VvC)*SUM[pSsCi] """

# tolerancia de los componentes (10%)
VvR=0.1
VvC=0.1

pr("Cambio por unidad para Wp")

Vvwp=VvR*(NUMwpStv[R1]+NUMwpStv[R2]+NUMwpStv[R3]) + VvC*(NUMwpStv[C1]+NUMwpStv[C2])

print(f"{Vvwp}\n")
pr()

pr("Cambio por unidad para BWp")

Vvbwp=VvR*(NUMbwpStv[R1]+NUMbwpStv[R2]+NUMbwpStv[R3]) + VvC*(NUMbwpStv[C1]+NUMbwpStv[C2])

print(f"{Vvbwp}\n")

--------------------------------------------------
           |Cambio por unidad para Wp|            
--------------------------------------------------
-0.200000000000000

****************************************************************************************************

----------------------------------------------------
            |Cambio por unidad para BWp|            
----------------------------------------------------
-0.202366167863213

